In [1]:
import os
import sys
import cv2
import csv
import tqdm
import pandas as pd
import random
import numpy as np
import time
from skimage.morphology import skeletonize
from matplotlib import pyplot as plt


In [10]:
##prior functions

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

def analyzeSORT(df,threshold):
    #threshold = 45
    vc = df.label.value_counts()
    test = vc[vc > threshold].index.tolist()
    csv_outputs = []
    deadboxes = []
    fullboxes = []
    deadcalls = []
  
    for ID in test:        
        filtval = df['label'] ==ID
        interim = df[filtval]
        interimD = []
        interim2 = np.asarray(interim)
        fill = 0
        deadcount = 0
        deathspots = []

        for row in interim2:
            frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
            if fill > 1:
                boxA = [x1A, y1A, x2A, y2A]
                boxB = [x1B, y1B, x2B, y2B]
                deltaA = bb_intersection_over_union(boxA, boxB) 
                #print(deltaA)
                if deltaA > 0.95:
                    deadcount += 1
                    #print(deadcount)
                #print(fill)
                if deadcount > 15:
                    catagoryA = 'dead'
                   # print(deadcount)
                if deadcount == 16:
                    if deadboxes == []:
                        deathspots.append([frameNA, x1A, y1A, x2A, y2A])     
                        deadboxes.append([x1A, y1A, x2A, y2A])  
                        csv_outputs.append((frameNA/72)+2)
                        deadcalls.append(labelA)
                    else:
                        notunique = 0
                        for box in deadboxes:
                            #print(box)
                            x1D, y1D, x2D, y2D, *_ = box
                            boxD = [x1D, y1D, x2D, y2D]
                            deltaD = bb_intersection_over_union(boxA, boxD)  
                            if deltaD > 0.3:
                                notunique = 1
                        if notunique == 0:
                            deathspots.append([frameNA, x1A, y1A, x2A, y2A])  
                            deadboxes.append([x1A, y1A, x2A, y2A])               
                            csv_outputs.append((frameNA/72)+2)

                    #print(deathtime)
                    #print(frameNA)
                #newRow = [frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA]
                #interimD.append(newRow)
            frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
            fullboxes.append([frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA])  

            fill +=1
            #if deadcount == 16:
                #break

    #csv_outputs = pd.DataFrame(csv_outputs, columns = ['#desc'])
    #csv_outputs['neural'] = '1'
    fullboxes = pd.DataFrame(fullboxes, columns = ['frame','x1','y1','x2','y2','label','delta','catagory'])

    return(fullboxes)

In [27]:
##new functions
def calcDistance(worm):
    totalDist = 0
    interval = 0
    for row in worm:
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if interval > 0:
            xabs = abs(x1A-x1B)
            yabs = abs(y1A-y1B)
            totalDist += np.hypot(xabs,yabs)
            
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        interval+=1
    return(totalDist)    

def calcMeanSpeed(worm):
    totalDist = 0
    interval = 0
    for row in worm:
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if interval > 0:
            xabs = abs(x1A-x1B)
            yabs = abs(y1A-y1B)
            totalDist += np.hypot(xabs,yabs)
            
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        interval+=1
    meanSpeed = totalDist/interval    
    return(meanSpeed) 

def calcAccels(worm):
    accelList = []
    interval = 0
    for  row in worm:
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if interval > 0:
            xabs = abs(x1A-x1B)
            yabs = abs(y1A-y1B)
            accelList.append(np.hypot(xabs,yabs))
            
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        interval+=1
    return(accelList) 

In [11]:
CSV_SORT_PATH = "D:/NN_alive_dead_training_vids/Sorted/344.csv"
df = pd.read_csv(CSV_SORT_PATH,names=('frame', 'x1', 'y1', 'x2', 'y2','label','delta'))
df['catagory'] = 'alive'
outputs = analyzeSORT(df,threshold = 75)

In [15]:

unique = outputs["label"].unique()
for uni in unique:
    filtval = outputs['label'] == uni
    csv_outputs = np.asarray(outputs[filtval])[:,:]

In [16]:
csv_outputs

array([[591.0, 1146.1312987221786, 445.207897161904, 1203.138656183865,
        468.6588147695233, 5383.0, nan, 'alive'],
       [592.0, 1140.2153458068567, 448.37261910907137,
        1200.2716301078192, 473.3379271435165, 5383.0, nan, 'alive'],
       [593.0, 1143.4219639896337, 447.77514406088414,
        1201.6671350744991, 471.132947276005, 5383.0, 0.8310821220154689,
        'alive'],
       [594.0, 1145.833429959051, 448.89994133260615, 1201.5233069751357,
        470.3976380976372, 5383.0, 0.8837969494739054, 'alive'],
       [595.0, 1145.115111438034, 450.6996843010497, 1200.9061806314269,
        470.7753000374653, 5383.0, 0.8847944055947914, 'alive'],
       [596.0, 1146.0108931519276, 448.74109078845055,
        1202.2351106734022, 469.3526323864437, 5383.0,
        0.8227882638712929, 'alive'],
       [600.0, 1141.629931668607, 453.66847470268766, 1197.1563898868787,
        471.47310015951854, 5383.0, 0.6085394116304338, 'alive'],
       [601.0, 1141.3875661071568, 453.99

In [31]:
print(calcDistance(csv_outputs))
print(calcMeanSpeed(csv_outputs))
print(np.mean(calcAccels(csv_outputs)))
print(np.max(calcAccels(csv_outputs)))

167.32671266476464
2.091583908309558
2.1180596539843632
7.147106341184914
